<font size=8>&nbsp;<font color=#3F7DCB>**q**<font color=#C9E9FF>**Bitlab =**</font> <font color=#F9AB00>**Colab**<font color=#C9E9FF> **+**<font color=#202229> **qBittorrent-nox**<font color=#C9E9FF>

###### In case you stumbled upon this colab notebook somewhere on the internet, here's the **[repo](https://github.com/hyPnOtICDo0g/Otakulab)**.

## Configuration
Upload your rclone config file and run this cell.

In [ ]:
#@markdown #### Fetch Dependencies
%%shell

if ! [ -e /content/rclone.conf ]
then
  echo -e "\e[31mRclone config not found."
  # using exit code 0 instead of 1 as colab tends to throw a ton of errors
  exit 0
fi

if ! [ -e /usr/bin/qbittorrent-nox ]
then
  mkdir -p /root/.config/qBittorrent
  curl -s https://pastebin.com/raw/mi1muHwS -o /root/.config/qBittorrent/qBittorrent.conf
  wget --show-progress -qO /usr/bin/qbittorrent-nox https://github.com/userdocs/qbittorrent-nox-static/releases/latest/download/x86_64-qbittorrent-nox
  chmod 700 /usr/bin/qbittorrent-nox
fi

rm -rf "/content/sample_data"
curl -s https://rclone.org/install.sh | sudo bash
npm -s install -g npm localtunnel
echo -e "\033[96mDependencies Fetched."

## Start a local qBittorrent server & port forward it using [*localtunnel*](https://theboroer.github.io/localtunnel-www)

In [ ]:
# @markdown <br><center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/66/New_qBittorrent_Logo.svg/2048px-New_qBittorrent_Logo.svg.png" height="80"></a></center><br>
from subprocess import Popen, PIPE
from pathlib import Path as path

if path('/usr/bin/qbittorrent-nox').exists():
  # start qBittorrent-nox as a daemon
  Popen(['qbittorrent-nox', '-d'])

  # start localtunnel & open a HTTPS tunnel on port 3000
  tunnel = Popen(['lt', '-p', '3000'], stdout=PIPE)
  url = tunnel.stdout.readline().decode('utf-8').replace('your url is: ', '').strip('\n')
  print('\033[92mWebUI:', url)
else:
  print('\033[91mqbittorrent-nox not found.')

## Transfer downloaded files using rclone 


In [ ]:
#@markdown #### • Enter the source & destination paths & run this cell.
#@markdown #### • You can obtain the **source path** by using the file manager to your left.
#@markdown #### • **Destination path** is the remote you want to transfer to.

SOURCE = "/content/Downloads" #@param {type:"string"}
DESTINATION = "drive:" #@param {type:"string"}
ARGS = "--transfers=10 --fast-list --stats-one-line -v" #@param {type:"string"}

!rclone copy "$SOURCE" "$DESTINATION" $ARGS --config rclone.conf

## Kill the server ☠️

*Omae wa mou shinderu.*

In [ ]:
from subprocess import run
STATUS = "alive" #@param ["alive", "ded"]
if STATUS == "ded":
  run(['pkill', '-9', 'qbittorrent-nox'])
  tunnel.terminate()